In [2]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [27]:
# setting up dataframe with columns required
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'link', 'rating'], index=None)

In [24]:
http = httplib2.Http()

In [ ]:
# the page links all follow pattern and can loop through nums to string just zero padded

# note from 2235 onwards there are a couple that do not have the standard codes in directory pages, will keep the standard code for now
# also some marked as missing from site or along those lines, probably added placeholder text, may want to add flag for easy filtering out

for n in range(1,7000):
  # timeout to limit load on the site (based on api requests per minute limit)
  if n % 240 == 0:
    print(n)
    time.sleep(60)

  # getting the number as a string in the way need to get url
  num_as_str = '{}{}'.format('0' * (3 - len(str(n))), str(n)) if n < 1000 else str(n)

  # requesting page for current/target scp, parsing with beautifulsoup
  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')

  # getting text for the scp within the main content div
  text = [i for i in soup.find(id = 'page-content').find_all('p')]

  # p tags include image captions which are excluded as below, may be other similar cases left in
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)

  # getting text from result set
  text = [i.text for i in text]

  # getting the rating - a few do not have this primarily the ones that are 'blocked' or similar
  rating = soup.find(class_ = 'number prw54353')
  if rating == None:
    pass
  else:
    rating = rating.text

  # accounting for different directory pages (used to get title) based on number - getting correct url below
  if n < 1000:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  else:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}'.format(str(int(n//1000)+1)))

  # requesting directory page
  soup = BeautifulSoup(response, 'html.parser')

  # some logic to get the text title (not scp number code) 
  # using next or previous siblings and numbers where needed as some have no code on the directory page or multiple codes
  if soup.find('a', text='SCP-{}'.format(num_as_str)) == None:
    # checks here due to cases where no previous or next sibling for one of the elements involved
    if soup.find('a', text='SCP-{}'.format(str(n-1))) != None and soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling() != None:
      title = soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling().text.split(" - ")[-1]
    else:
      title = soup.find('a', text='SCP-{}'.format(str(n+1))).parent.find_previous_sibling().text
      # removing the code part of titles that do have a code here
      if ' - ' in title:
        title = title.split(" - ")[-1]
    # could potentially add both codes for those that have two
  # basic title finder for most cases
  else:
    title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # quotes to preserve commas and not cause csv issues - joining up p tags with newlines
  text = "\"{}\"".format(" \n ".join(i for i in text))
  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), text, url, rating]
  scp_main_df.loc[len(scp_main_df)] = info



In [ ]:
scp_main_df.head

In [ ]:
scp_main_df.to_csv('scp6999.csv', index=False) 

In [19]:
# scp_df = pd.read_csv('scp6999.csv', delimiter=',', quotechar='"')
# scp_df.columns = ['index', 'code', 'title', 'text', 'link', 'rating']


In [20]:
# # some extra light  processing
# scp_df['text'] = scp_df['text'].apply(lambda x: "\"{}\"".format(" \n ".join(i.strip("\'\"") for i in x[2:-2].split(', '))))


In [22]:
# scp_df.to_csv('scpjoinedtext.csv')

In [ ]:
# print(scp_df['text'][1])
# # checking how the join method ended up